In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

In [2]:
this_dir = Path.cwd()
dataset_file = this_dir / "data/pd_speech_features.ods"
csv_dir = this_dir / "plots/csvs"

In [3]:
df = pd.read_excel(dataset_file, header=[0, 1], engine="odf")
df.head()

Basic Info        Baseline Features                              \
          id gender               PPE      DFA     RPDE numPulses   
0          0      1           0.85247  0.71826  0.57227       240   
1          0      1           0.76686  0.69481  0.53966       234   
2          0      1           0.85083  0.67604  0.58982       232   
3          1      0           0.41121  0.79672  0.59257       178   
4          1      0           0.32790  0.79782  0.53028       236   

                                                                     ...  \
  numPeriodsPulses meanPeriodPulses stdDevPeriodPulses locPctJitter  ...   
0              239         0.008064           0.000087      0.00218  ...   
1              233         0.008258           0.000073      0.00195  ...   
2              231         0.008340           0.000060      0.00176  ...   
3              177         0.010858           0.000183      0.00419  ...   
4              235         0.008162           0.002669      0.00535  ...   

              TQWT Features                            \
  tqwt_kurtosisValue_dec_28 tqwt_kurtosisValue_dec_29   
0                    1.5620                    2.6445   
1                    1.5589                    3.6107   
2                    1.5643                    2.3308   
3                    3.7805                    3.5664   
4                    6.1727                    5.8416   

                                                       \
  tqwt_kurtosisValue_dec_30 tqwt_kurtosisValue_dec_31   
0                    3.8686                    4.2105   
1                   23.5155                   14.1962   
2                    9.4959                   10.7458   
3                    5.2558                   14.0403   
4                    6.0805                    5.7621   

                                                       \
  tqwt_kurtosisValue_dec_32 tqwt_kurtosisValue_dec_33   
0                    5.1221                    4.4625   
1                   11.0261                    9.5082   
2                   11.0177                    4.8066   
3                    4.2235                    4.6857   
4                    7.7817                   11.6891   

                                                       \
  tqwt_kurtosisValue_dec_34 tqwt_kurtosisValue_dec_35   
0                    2.6202                    3.0004   
1                    6.5245                    6.3431   
2                    2.9199                    3.1495   
3                    4.8460                    6.2650   
4                    8.2103                    5.0559   

                            Class  
  tqwt_kurtosisValue_dec_36 class  
0                   18.9405     1  
1                   45.1780     1  
2                    4.7666     1  
3                    4.0603     1  
4                    6.1164     1  

[5 rows x 755 columns]

In [10]:
models = [c.stem for c in csv_dir.glob("*.csv")]

In [11]:
models, len(models)

(['AdaBoostClassifier',
  'DecisionTreeClassifier',
  'GaussianNB',
  'GradientBoostingClassifier',
  'KNeighborsClassifier',
  'LogisticRegression',
  'MLP',
  'RandomForestClassifier',
  'SVC',
  'tf_self_attention'],
 10)

In [12]:
model_dfs = {c.stem: pd.read_csv(c) for c in csv_dir.glob("*.csv")}

In [37]:
hard_sample_dict = {}
for model in models:
    dfm = model_dfs[model]
    hard_sample = dfm[dfm['Match'] == False]
    hard_sample_ids = list(hard_sample['UID'])
    hard_sample_dict[model] = set([int(h) for h in hard_sample_ids])
    print(model, "\t",len(hard_sample_ids))

AdaBoostClassifier 	 105
DecisionTreeClassifier 	 142
GaussianNB 	 150
GradientBoostingClassifier 	 101
KNeighborsClassifier 	 115
LogisticRegression 	 130
MLP 	 80
RandomForestClassifier 	 108
SVC 	 101
tf_self_attention 	 77


In [46]:
global_hard_samples = hard_sample_dict['GaussianNB']

for m, hs in hard_sample_dict.items():
#     print(m, len(hs), type(hs))
#     print(hs)
    global_hard_samples = global_hard_samples.intersection(hs)

In [47]:
len(global_hard_samples)

16

In [48]:
global_hard_samples

{43, 44, 72, 82, 128, 236, 249, 310, 334, 461, 490, 518, 700, 702, 723, 727}

In [70]:
unioun_hard_sample = hard_sample_dict['AdaBoostClassifier']
ignore_model = ['GaussianNB', 'LogisticRegression', 'DecisionTreeClassifier']
for m, hs in hard_sample_dict.items():
    if m in ignore_model:
        continue
    unioun_hard_sample = unioun_hard_sample.union(hs)

In [72]:
len(unioun_hard_sample)

244

In [53]:
tf_mlp = hard_sample_dict['MLP'].intersection(hard_sample_dict['tf_self_attention'])

In [54]:
len(tf_mlp)

51

In [62]:
tree = hard_sample_dict['AdaBoostClassifier'].intersection(hard_sample_dict['GradientBoostingClassifier']).intersection(hard_sample_dict['RandomForestClassifier'])

In [63]:
len(tree)

42

In [75]:
len(tf_mlp.intersection(tree).intersection(hard_sample_dict['SVC']))

29

In [74]:
df.loc[global_hard_samples]

Basic Info        Baseline Features                              \
            id gender               PPE      DFA     RPDE numPulses   
128         42      1           0.81634  0.64393  0.57462       273   
518        172      0           0.79190  0.59987  0.30515       429   
72          24      0           0.16874  0.71210  0.61151       166   
490        163      1           0.78590  0.70792  0.34619       295   
43          14      0           0.88352  0.79509  0.36585       397   
44          14      0           0.86478  0.76570  0.37052       400   
461        153      1           0.85127  0.69896  0.56694       232   
334        111      1           0.82292  0.70036  0.55794       237   
236         78      1           0.80997  0.68753  0.55747       237   
82          27      0           0.84157  0.72248  0.36795       441   
723        241      0           0.77200  0.56362  0.39992       442   
310        103      0           0.36923  0.75809  0.66084       192   
727        242      1           0.80880  0.75498  0.51913       313   
249         83      0           0.86080  0.71266  0.40057       460   
700        233      0           0.89604  0.64010  0.27293       395   
702        234      0           0.79305  0.75417  0.67653       312   

                                                                       ...  \
    numPeriodsPulses meanPeriodPulses stdDevPeriodPulses locPctJitter  ...   
128              272         0.007080           0.000136      0.00179  ...   
518              428         0.004496           0.000058      0.00093  ...   
72               165         0.011639           0.000158      0.00252  ...   
490              294         0.006555           0.000050      0.00136  ...   
43               396         0.004861           0.000054      0.00143  ...   
44               399         0.004827           0.000054      0.00140  ...   
461              231         0.008314           0.000083      0.00148  ...   
334              236         0.008157           0.000088      0.00244  ...   
236              236         0.008012           0.000448      0.00370  ...   
82               440         0.004372           0.000038      0.00061  ...   
723              441         0.004364           0.001407      0.00157  ...   
310              191         0.010083           0.000218      0.00302  ...   
727              312         0.006154           0.000279      0.00169  ...   
249              459         0.004194           0.000042      0.00126  ...   
700              394         0.004888           0.000017      0.00048  ...   
702              311         0.006192           0.000329      0.00352  ...   

                TQWT Features                            \
    tqwt_kurtosisValue_dec_28 tqwt_kurtosisValue_dec_29   
128                    3.6303                  102.5030   
518                   69.3162                   65.4094   
72                     2.7684                    2.5700   
490                    4.8595                    3.3476   
43                     1.5935                    2.2826   
44                     1.5610                    4.7079   
461                    1.6378                    9.5337   
334                    6.5239                    5.1212   
236                    4.9903                    5.8904   
82                     3.7369                    2.5475   
723                   75.3768                   42.2440   
310                    4.8060                    4.7118   
727                  173.1197                  136.4235   
249                    2.2236                   59.4541   
700                   42.7013                   34.2372   
702                    5.1837                    8.2270   

                                                         \
    tqwt_kurtosisValue_dec_30 tqwt_kurtosisValue_dec_31   
128                   35.4456                   32.6962   
518                   29.6029                    7.3709   
72                   